In [1]:
import os
import sys
import glob
import pickle

os.environ['CUDA_VISIBLE_DEVICES'] = '4'

sys.path.insert(0, '../src-py/')

In [2]:
import bert_moral_classification
import utils
import json
import traceback
import pandas as pd

import numpy as np
from numpy import cov
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
from scipy.stats import spearmanr

In [3]:
df = pd.read_json('/workspace/ceph_data/moral-based-argumentation/editorials-18/corpus-webis-editorial-quality-18/corpus-webis-editorial-quality-18.json')

In [74]:
df.head()

,annotator_id,article_id,political_typology,effect,explanation,article_index,change,empower,date_created,political_pole,effect_abstracted,batch,intensity,article_paras,article_morals,article_moral_counts
0,C03,1638699.txt,Country First Conservatives,5,I have made public resolutions that failed and...,0,None,YES,2018-05-04 18:34:14.588362+02,conservative,3,batch1,strong,"[No matter how you look at it, the new year is...","[{authority, purity}, {loyalty, authority}, {a...","{'authority': 1.0, 'purity': 0.5, 'loyalty': 0..."
1,C03,1640113.txt,Country First Conservatives,5,This is one of those that I wouldn’t argue bec...,1,None,NO,2018-05-04 18:34:14.588362+02,conservative,3,batch1,strong,[It was nice to hear Alberto Gonzales tell the...,"[{authority, fairness}, {}, {authority}, {auth...","{'authority': 0.75, 'fairness': 0.5}"
2,C03,1640630.txt,Country First Conservatives,3,"River restoration is nice, I suppose.",2,None,None,2018-05-04 18:34:14.588362+02,conservative,2,batch1,none,"[In 1998, with great expectations and after ye...","[{care, authority}, {care, authority}, {care, ...","{'care': 0.625, 'authority': 0.75}"
3,C03,1640915.txt,Country First Conservatives,2,"I feel like we should be helping the poor, but...",3,None,None,2018-05-04 18:34:14.588362+02,conservative,1,batch1,moderate,[Kofi Annan has two problems -- the Bush admin...,"[{fairness}, {authority}, {care, authority}, {...","{'fairness': 0.3333333333333333, 'authority': ..."
4,L07,1638699.txt,Disaffected Democrats,4,I genuinely believe that people need to be act...,0,None,None,2018-05-04 20:24:51.224554+02,liberal,3,batch1,moderate,"[No matter how you look at it, the new year is...","[{authority, purity}, {loyalty, authority}, {a...","{'authority': 1.0, 'purity': 0.5, 'loyalty': 0..."


In [45]:
def load_txts(path = '/workspace/ceph_data/moral-based-argumentation/editorials-18/elbaff_editorials_18/corpus/*'):
    docs = {}
    for f in glob.glob(path):
        docs[f.split('/')[-1]] = open(f, encoding='utf-8').read()
    return docs

In [47]:
#editorials_txts = load_txts()

In [7]:
df['article_paras'] = df.article_id.apply(lambda x: editorials_txts[x].split('\n'))

In [ ]:
df['article_morals'] = df.article_paras.apply(lambda paras: bert_moral_classification.get_arg_morals(paras))

In [164]:
df.to_pickle('/workspace/ceph_data/moral-based-argumentation/editorials-18/elbaff_editorials_18/editorials_with_morals.pkl')

-------------

1: Strongly Challenging

2: Somewhat Challenging

3: No Effect

4: Somewhat Reinforcing

5: Strongly Reinforcing

In [4]:
def compute_moral_load(para_morals):
    all_morals = [moral for morals in para_morals for moral in morals]
    moral_counts = dict(Counter(all_morals))
    moral_perc = {x[0]: x[1]/len(para_morals) for x in moral_counts.items()}
    
    return moral_perc

In [5]:
def disply_correlations(morals, target):
    variables = np.array([x + [y] for x, y in zip(morals, target)])
    corr, p = spearmanr(variables[:,0], variables[:,5])
    print("CARE: Corr is  {} and P-value: {}".format(round(corr, 3), round(p, 3)))
    corr, p = spearmanr(variables[:,1], variables[:,5])
    print("FAIRNESS: Corr is  {} and P-value: {}".format(round(corr, 3), round(p, 3)))
    corr, p = spearmanr(variables[:,2], variables[:,5])
    print("AUTHORITY: Corr is {} and P-value: {}".format(round(corr, 3), round(p, 3)))
    corr, p = spearmanr(variables[:,3], variables[:,5])
    print("LOYALTY: Corr is  {} and P-value: {}".format(round(corr, 3), round(p, 3)))
    corr, p = spearmanr(variables[:,4], variables[:,5])
    print("PURITY: Corr is  {} and P-value: {}".format(round(corr, 3), round(p, 3)))

In [6]:
df = pd.read_pickle('/workspace/ceph_data/moral-based-argumentation/editorials-18/elbaff_editorials_18/editorials_with_morals.pkl')

In [7]:
from collections import Counter

In [8]:
df['article_moral_counts'] = df.article_morals.apply(lambda x: compute_moral_load(x))

In [17]:
int_map = {
    'strong':2,
    'moderate':1,
    'none':0
}

#### Effect on Conservatives:

In [18]:
cons_effect = df[(df.political_pole == 'conservative')]['intensity'].tolist()
cons_effect = [int_map[x] for x in cons_effect]
cons_morals = df[(df.political_pole == 'conservative')]['article_moral_counts'].tolist()
cons_morals = [[d[m] if m in d else 0 for m in ['care', 'fairness', 'authority', 'loyalty', 'purity']] for d in cons_morals]

In [9]:
# cons_effect = df[(df.political_pole == 'conservative') & (df.effect.isin([1, 2, 3]))]['effect'].tolist()
# cons_effect = [1 if x in [1, 2] else 0  for x in cons_effect]
# cons_morals = df[(df.political_pole == 'conservative') & (df.effect.isin([1, 2, 3]))]['article_moral_counts'].tolist()
# cons_morals = [[d[m] if m in d else 0 for m in ['care', 'fairness', 'authority', 'loyalty', 'purity']] for d in cons_morals]

In [20]:
disply_correlations(cons_morals, cons_effect)

CARE: Corr is  0.052 and P-value: 0.005
FAIRNESS: Corr is  0.003 and P-value: 0.873
AUTHORITY: Corr is -0.008 and P-value: 0.654
LOYALTY: Corr is  0.008 and P-value: 0.66
PURITY: Corr is  0.004 and P-value: 0.837


#### Effect on Liberals:

In [14]:
len(lib_effect)

2660

In [12]:
lib_effect = df[(df.political_pole == 'liberal') & (df.effect.isin([3, 4, 5]))]['effect'].tolist()
lib_effect = [1 if x in [4, 5] else 0  for x in lib_effect]
lib_morals = df[(df.political_pole == 'liberal') & (df.effect.isin([3, 4, 5]))]['article_moral_counts'].tolist()
lib_morals = [[d[m] if m in d else 0 for m in ['care', 'fairness', 'authority', 'loyalty', 'purity']] for d in lib_morals]

In [13]:
disply_correlations(lib_morals, lib_effect)

CARE: Corr is  0.055 and P-value: 0.005
FAIRNESS: Corr is  0.026 and P-value: 0.184
AUTHORITY: Corr is -0.02 and P-value: 0.299
LOYALTY: Corr is  -0.017 and P-value: 0.373
PURITY: Corr is  -0.024 and P-value: 0.213


-------------

### Correlation between morals and Changing the stance:

In [60]:
changed_df = df[(df.change=='YES') | (df.change=='NO')].copy()
changed_df['change'] = changed_df.change.apply(lambda x: 1 if x=='YES' else 0)

In [61]:
changed_df.change.value_counts()

0    5932
1      68
Name: change, dtype: int64

In [62]:
changed_df.political_pole.value_counts()

liberal         3000
conservative    3000
Name: political_pole, dtype: int64

#### On Conservatives:

In [63]:
changed = changed_df[(changed_df.political_pole == 'conservative')]['change'].tolist()
cons_morals = changed_df[(changed_df.political_pole == 'conservative')]['article_moral_counts'].tolist()
cons_morals = [[d[m] if m in d else 0 for m in ['care', 'fairness', 'authority', 'loyalty', 'purity']] for d in cons_morals]

In [64]:
disply_correlations(cons_morals, changed)

CARE: Corr is  0.049 and P-value: 0.008
FAIRNESS: Corr is  -0.038 and P-value: 0.038
AUTHORITY: Corr is -0.021 and P-value: 0.24
LOYALTY: Corr is  -0.012 and P-value: 0.525
PURITY: Corr is  0.03 and P-value: 0.105


#### On Liberals:

In [65]:
changed = changed_df[(changed_df.political_pole == 'liberal')]['change'].tolist()
lib_morals = changed_df[(changed_df.political_pole == 'liberal')]['article_moral_counts'].tolist()
lib_morals = [[d[m] if m in d else 0 for m in ['care', 'fairness', 'authority', 'loyalty', 'purity']] for d in lib_morals]

In [66]:
disply_correlations(lib_morals, changed)

CARE: Corr is  0.008 and P-value: 0.656
FAIRNESS: Corr is  -0.019 and P-value: 0.305
AUTHORITY: Corr is -0.014 and P-value: 0.437
LOYALTY: Corr is  -0.03 and P-value: 0.104
PURITY: Corr is  -0.017 and P-value: 0.362


---------------

### Correlation between morals and Empowering the stance:

In [67]:
empower_df = df[(df.empower=='YES') | (df.empower=='NO')].copy()
empower_df['empower'] = empower_df.empower.apply(lambda x: 1 if x=='YES' else 0)

In [68]:
empower_df.empower.value_counts()

0    5030
1     970
Name: empower, dtype: int64

In [69]:
empower_df.political_pole.value_counts()

liberal         3000
conservative    3000
Name: political_pole, dtype: int64

#### On Conservatives:

In [70]:
empower = empower_df[(empower_df.political_pole == 'conservative')]['empower'].tolist()
cons_morals = empower_df[(empower_df.political_pole == 'conservative')]['article_moral_counts'].tolist()
cons_morals = [[d[m] if m in d else 0 for m in ['care', 'fairness', 'authority', 'loyalty', 'purity']] for d in cons_morals]

In [71]:
disply_correlations(cons_morals, empower)

CARE: Corr is  0.01 and P-value: 0.591
FAIRNESS: Corr is  0.027 and P-value: 0.139
AUTHORITY: Corr is 0.01 and P-value: 0.586
LOYALTY: Corr is  0.031 and P-value: 0.086
PURITY: Corr is  0.017 and P-value: 0.34


#### On Liberals:

In [72]:
empower = empower_df[(empower_df.political_pole == 'liberal')]['empower'].tolist()
liberal_morals = empower_df[(empower_df.political_pole == 'liberal')]['article_moral_counts'].tolist()
liberal_morals = [[d[m] if m in d else 0 for m in ['care', 'fairness', 'authority', 'loyalty', 'purity']] for d in liberal_morals]

In [73]:
disply_correlations(liberal_morals, empower)

CARE: Corr is  0.003 and P-value: 0.853
FAIRNESS: Corr is  0.045 and P-value: 0.014
AUTHORITY: Corr is -0.01 and P-value: 0.602
LOYALTY: Corr is  -0.001 and P-value: 0.977
PURITY: Corr is  -0.014 and P-value: 0.455
